<a href="https://colab.research.google.com/github/AnilaGibson/PortfolioProjects/blob/main/AIMentalHealthChatBox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd #For Data Tables
import numpy as np #Number crunching
import re #For text cleaning with regular expressions
import matplotlib.pyplot as plt
import seaborn as sns #Statistical visualizations
#For machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
import nltk #Process human language
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import punkt # break sentences into words
from nltk import wordnet #Lemmatization
import warnings
import joblib #Saving the trained model
# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Download necessary NLTK resources
print("Setting up NLTK resources...")
try:
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('wordnet', quiet=True)
except:
   print("Note: Some NLTK resources could not be downloaded. Basic text processing will still work.")


Setting up NLTK resources...


In [ ]:
#Data Loading and Exploration
## This section loads the dataset,
#explores its structure, and handles missing values.
from google.colab import files
oploaded = files.upload()
print("Loading the dataset...")
try:
    # Load the dataset with the correct column names
    data = pd.read_csv("Combined Data.csv")
    print("Dataset loaded successfully!")

    # Print the actual column names to verify
    print(f"Actual column names in the dataset: {data.columns.tolist()}")

    # Map column names to expected format for consistency in code
    statement_col = 'statement'  # The actual column name for statements
    status_col = 'status'        # The actual column name for mental health status


except FileNotFoundError:
    print("Error: 'Combined Data.csv' file not found. Please make sure the file is in the correct location.")
    # Create a sample dataset for demonstration if the file is not found
    print("Creating a sample dataset for demonstration purposes.")
    # Fix: Ensure all lists in sample_data have the same length
    num_statements = len([
        # Normal statements
        "I'm feeling great today, excited about the future.",
        "Just finished a great workout and feeling positive.",
        "Had a wonderful time with friends tonight, feeling connected.",
        "I accomplished all my goals for the day and feel satisfied.",
        "The weather is beautiful and I had a nice walk outside.",
        "I'm enjoying my new hobby and learning a lot.",
        "Spent quality time with family this weekend, feeling grateful.",
        "Made progress on my personal project today, feeling accomplished.",
        "I've been eating healthy and noticing positive changes.",
        "Starting my day with meditation really helps me stay focused.",
        # Depression statements
        "I feel so empty and worthless all the time.",
        "I can't seem to enjoy things that used to make me happy.",
        "I've been sleeping too much but still feel exhausted.",
        "Everything feels like such an effort lately.",
        "I feel like a burden to everyone around me.",
            "I haven't had any appetite for days.",
            "Nothing matters anymore, I feel numb to everything.",
            "I can't concentrate on anything, my mind feels foggy.",
            "I've isolated myself from everyone for weeks now.",
            "Even small tasks feel overwhelming and impossible.",

            # Suicidal statements
            "I don't want to be alive anymore, there's no point.",
            "I've been planning my suicide for weeks now.",
            "Everyone would be better off without me.",
            "I can't see any way out of this pain except ending it all.",
            "I've written my goodbye letters already.",
            "There's nothing left for me in this world.",
            "I keep thinking about ways to end my life.",
            "The pain is too much to bear, I just want it to stop.",
            "I've given away my important possessions.",
            "No one would notice if I disappeared forever.",

            # Anxiety statements
            "I'm having trouble sleeping and constantly worried about everything.",
            "My heart races and I feel like I'm dying when I go outside.",
            "I can't stop checking things, it's exhausting.",
            "I worry about the smallest things all the time.",
            "My mind won't stop racing with worst-case scenarios.",
            "I keep having panic attacks out of nowhere.",
            "I'm constantly on edge waiting for something bad to happen.",
            "Social situations make me feel like everyone is judging me.",
            "I overthink every conversation I have had.",
            "I can't make decisions because I'm afraid of making mistakes.",

            # Stress statements
            "I'm overwhelmed with work and can't keep up.",
            "Work is stressing me out so much lately.",
            "The pressure is unbearable, I can't handle it.",
            "I can't focus on anything, my mind is racing.",
            "I feel like I'm being pulled in too many directions at once.",
            "My to-do list keeps growing and I can't catch up.",
            "I haven't had a moment to relax in weeks.",
            "The deadlines are making me physically ill.",
            "I'm grinding my teeth at night from all the tension.",
            "I snap at people because I'm under so much pressure.",

            # Bi-Polar statements
            "Sometimes I'm so energetic, then I crash into deep sadness.",
            "One minute I'm on top of the world, the next I want to die.",
            "My emotions swing wildly and I can't control them.",
            "I went on a shopping spree and maxed out my credit cards, now I regret it.",
            "I haven't slept for three days but feel amazing and full of energy.",
            "I've been making grand plans but then abandon them when I crash.",
            "People say I talk too fast when I'm having a good week.",
            "I switch between feeling invincible and completely worthless.",
            "Last month I was depressed, this week I'm incredibly productive.",
            "I make impulsive decisions when I feel good that I later regret.",

            # Personality Disorder statements
            "I don't trust anyone, they all have hidden motives.",
            "I feel like I have multiple personalities sometimes.",
            "My relationships are either perfect or terrible, nothing in between.",
            "I'm terrified of being abandoned by everyone.",
            "I don't know who I really am, my identity keeps shifting.",
            "I see things in black and white, people are either with me or against me.",
            "I manipulate others to get what I need.",
            "I engage in risky behaviors just to feel something.",
            "I have intense relationships that don't last.",
        "I feel empty inside and don't know my purpose in life."
    ])

    sample_data = {
        'unique_id': list(range(1, num_statements + 1)),
        'statement': [
            # Normal statements
            "I'm feeling great today, excited about the future.",
            "Just finished a great workout and feeling positive.",
            "Had a wonderful time with friends tonight, feeling connected.",
            "I accomplished all my goals for the day and feel satisfied.",
            "The weather is beautiful and I had a nice walk outside.",
            "I'm enjoying my new hobby and learning a lot.",
            "Spent quality time with family this weekend, feeling grateful.",
            "Made progress on my personal project today, feeling accomplished.",
            "I've been eating healthy and noticing positive changes.",
            "Starting my day with meditation really helps me stay focused.",

            # Depression statements
            "I feel so empty and worthless all the time.",
            "I can't seem to enjoy things that used to make me happy.",
            "I've been sleeping too much but still feel exhausted.",
            "Everything feels like such an effort lately.",
            "I feel like a burden to everyone around me.",
            "I haven't had any appetite for days.",
            "Nothing matters anymore, I feel numb to everything.",
            "I can't concentrate on anything, my mind feels foggy.",
            "I've isolated myself from everyone for weeks now.",
            "Even small tasks feel overwhelming and impossible.",

            # Suicidal statements
            "I don't want to be alive anymore, there's no point.",
            "I've been planning my suicide for weeks now.",
            "Everyone would be better off without me.",
            "I can't see any way out of this pain except ending it all.",
            "I've written my goodbye letters already.",
            "There's nothing left for me in this world.",
            "I keep thinking about ways to end my life.",
            "The pain is too much to bear, I just want it to stop.",
            "I've given away my important possessions.",
            "No one would notice if I disappeared forever.",

            # Anxiety statements
            "I'm having trouble sleeping and constantly worried about everything.",
            "My heart races and I feel like I'm dying when I go outside.",
            "I can't stop checking things, it's exhausting.",
            "I worry about the smallest things all the time.",
            "My mind won't stop racing with worst-case scenarios.",
            "I keep having panic attacks out of nowhere.",
            "I'm constantly on edge waiting for something bad to happen.",
            "Social situations make me feel like everyone is judging me.",
            "I overthink every conversation I have had.",
            "I can't make decisions because I'm afraid of making mistakes.",

            # Stress statements
            "I'm overwhelmed with work and can't keep up.",
            "Work is stressing me out so much lately.",
            "The pressure is unbearable, I can't handle it.",
            "I can't focus on anything, my mind is racing.",
            "I feel like I'm being pulled in too many directions at once.",
            "My to-do list keeps growing and I can't catch up.",
            "I haven't had a moment to relax in weeks.",
            "The deadlines are making me physically ill.",
            "I'm grinding my teeth at night from all the tension.",
            "I snap at people because I'm under so much pressure.",

            # Bi-Polar statements
            "Sometimes I'm so energetic, then I crash into deep sadness.",
            "One minute I'm on top of the world, the next I want to die.",
            "My emotions swing wildly and I can't control them.",
            "I went on a shopping spree and maxed out my credit cards, now I regret it.",
            "I haven't slept for three days but feel amazing and full of energy.",
            "I've been making grand plans but then abandon them when I crash.",
            "People say I talk too fast when I'm having a good week.",
            "I switch between feeling invincible and completely worthless.",
            "Last month I was depressed, this week I'm incredibly productive.",
            "I make impulsive decisions when I feel good that I later regret.",

            # Personality Disorder statements
            "I don't trust anyone, they all have hidden motives.",
            "I feel like I have multiple personalities sometimes.",
            "My relationships are either perfect or terrible, nothing in between.",
            "I'm terrified of being abandoned by everyone.",
            "I don't know who I really am, my identity keeps shifting.",
            "I see things in black and white, people are either with me or against me.",
            "I manipulate others to get what I need.",
            "I engage in risky behaviors just to feel something.",
            "I have intense relationships that don't last.",
            "I feel empty inside and don't know my purpose in life."
        ],
        'status': [
            # Mental health status labels matching the statements above
            'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
            'Normal', 'Normal', 'Normal', 'Normal', 'Normal',

            'Depression', 'Depression', 'Depression', 'Depression', 'Depression',
            'Depression', 'Depression', 'Depression', 'Depression', 'Depression',

            'Suicidal', 'Suicidal', 'Suicidal', 'Suicidal', 'Suicidal',
            'Suicidal', 'Suicidal', 'Suicidal', 'Suicidal', 'Suicidal',

            'Anxiety', 'Anxiety', 'Anxiety', 'Anxiety', 'Anxiety',
            'Anxiety', 'Anxiety', 'Anxiety', 'Anxiety', 'Anxiety',

            'Stress', 'Stress', 'Stress', 'Stress', 'Stress',
            'Stress', 'Stress', 'Stress', 'Stress', 'Stress',

            'Bi-Polar', 'Bi-Polar', 'Bi-Polar', 'Bi-Polar', 'Bi-Polar',
            'Bi-Polar', 'Bi-Polar', 'Bi-Polar', 'Bi-Polar', 'Bi-Polar',

            'Personality Disorder', 'Personality Disorder', 'Personality Disorder', 'Personality Disorder', 'Personality Disorder',
            'Personality Disorder', 'Personality Disorder', 'Personality Disorder', 'Personality Disorder', 'Personality Disorder'
        ]
    }
    data = pd.DataFrame(sample_data)
    statement_col = 'statement'
    status_col = 'status'

# Display basic information about the dataset
print("\nDataset information:")
print(f"Number of entries: {data.shape[0]}")
print(f"Number of features: {data.shape[1]}")
print("\nFirst 5 rows of the dataset:")
print(data.head())

# Check for missing values
print("\nMissing values in each column:")
print(data.isnull().sum())

#remove rows with missing values
data_cleaned = data.dropna()
print("\nAfter removing rows with missing values:")
print(f"Number of entries after cleaning: {data_cleaned.shape[0]}")

# Basic statistics about the dataset
print("\nDistribution of Mental Health Status:")
print(data[status_col].value_counts())

# Visualize the distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=data, x=status_col)
plt.title('Distribution of Mental Health Status')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('mental_health_distribution.png')
plt.close()

Saving Combined Data.csv to Combined Data (3).csv
Loading the dataset...
Dataset loaded successfully!
Actual column names in the dataset: ['Unnamed: 0', 'statement', 'status']

Dataset information:
Number of entries: 53043
Number of features: 3

First 5 rows of the dataset:
   Unnamed: 0                                          statement   status
0           0                                         oh my gosh  Anxiety
1           1  trouble sleeping, confused mind, restless hear...  Anxiety
2           2  All wrong, back off dear, forward doubt. Stay ...  Anxiety
3           3  I've shifted my focus to something else but I'...  Anxiety
4           4  I'm restless and restless, it's been a month n...  Anxiety

Missing values in each column:
Unnamed: 0      0
statement     362
status          0
dtype: int64

After removing rows with missing values:
Number of entries after cleaning: 52681

Distribution of Mental Health Status:
status
Normal                  16351
Depression              

In [ ]:
#Text Preprocessing

def preprocess_text(text):
    """
    Clean and preprocess text data for analysis with fallback options
    Parameters:
    - text (str): The text to be processed

    Returns:
    - str: The processed text
    """
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()

        # Remove special characters and digits
        text = re.sub(r"[^a-zA-Z\s]", "", text)

        # Tokenization- splits the sentences into words
        try:
            tokens = nltk.word_tokenize(text)
        except:
            # Simple fallback tokenization
            tokens = text.split()

        # Remove stopwords - with fallback
        try:
            stop_words = set(stopwords.words('english'))
            filtered_tokens = [word for word in tokens if word not in stop_words] #Remove common words like "the, "is", "and"
        except:
            # Common English stopwords as fallback
            common_stopwords = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
                             'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him',
                             'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its',
                             'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what',
                             'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am',
                             'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has',
                             'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
                             'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of',
                             'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
                             'through', 'during', 'before', 'after', 'above', 'below', 'to',
                             'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under',
                             'again', 'further', 'then', 'once', 'here', 'there', 'when',
                             'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few',
                             'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
                             'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't',
                             'can', 'will', 'just', 'done', 'should', 'now'}
            filtered_tokens = [word for word in tokens if word not in common_stopwords]

        # Lemmatization, reduce words to their dictionary form, known as lemma,
        #Helps group together different forms of the same word to improve accuracy
        #and efficiency of text analysis
        #for example, "running" and "ran" are all different forms of the verb run
        try:
            lemmatizer = WordNetLemmatizer()
            lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
        except Exception as e:
            print(f"Lemmatization error: {e}. Using tokens without lemmatization.")
            lemmatized_tokens = filtered_tokens

        # Join tokens back into text
        processed_text = ' '.join(lemmatized_tokens)
        return processed_text
    else:
        return ""

print("\nPreprocessing text data...")
# Apply preprocessing to the statement column
data['Processed_Statement'] = data[statement_col].apply(preprocess_text)

print("Sample of preprocessed text:")
for i in range(min(3, len(data))):
    print(f"Original: {data[statement_col].iloc[i]}")
    print(f"Processed: {data['Processed_Statement'].iloc[i]}")
    print("---")


Preprocessing text data...
Sample of preprocessed text:
Original: oh my gosh
Processed: oh gosh
---
Original: trouble sleeping, confused mind, restless heart. All out of tune
Processed: trouble sleeping confused mind restless heart tune
---
Original: All wrong, back off dear, forward doubt. Stay in a restless and restless place
Processed: wrong back dear forward doubt stay restless restless place
---


In [ ]:
#Feature Engineering

# Create TF-IDF vectorizer- turns text into number, top 1000 words to focus on the
#words that are likely to be most helpful in classifying the mental health status
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# Split data into training and testing sets
X = data['Processed_Statement'] #cleaned text
y = data[status_col] #mental health label (output)

try:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) #80train 20test split
    print("\nData split into training and testing sets:")
    print(f"Training set size: {X_train.shape[0]}")
    print(f"Testing set size: {X_test.shape[0]}")
except ValueError as e:
    print(f"Error splitting data: {e}")
    print("Using the entire dataset for training.")
    X_train, y_train = X, y
    X_test, y_test = X.iloc[:10], y.iloc[:10]  # Just use a small subset for testing


Data split into training and testing sets:
Training set size: 42434
Testing set size: 10609


In [ ]:
#Model Building and Training

# Create a pipeline with TF-IDF and Logistic Regression
#it converts text input to numerical data
#then it uses a logistic regression model to learn from that data
#make predictions on the mental health status based on the patterns it has learned.
print("\nBuilding and training the model...")
model_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('classifier', LogisticRegression(max_iter=1000, C=1.0, solver='liblinear'))
])

# Train the model


try:
    model_pipeline.fit(X_train, y_train) #model trains on the training data
    print("Model trained successfully!")

    # Evaluate the model
#takes the trained model, uses it to make predictions on the test data,
#and then compares those predictions to the actual answers to see how
#often the model is correct. The result of this comparison is the model's accuracy
    y_pred = model_pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nModel accuracy: {accuracy:.2f}")


#Precision: Out of all the statements the model classified as belonging to a certain mental health status, how many were actually correct?
#Recall: Out of all the statements that truly belong to a certain mental health status, how many did the model correctly identify?
#F1-score: A balanced measure that considers both precision and recall.
#Support: The number of actual statements in each mental health status category.

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Save the model
    joblib.dump(model_pipeline, 'mental_health_model.pkl') #saves the model for later
    print("Model saved as 'mental_health_model.pkl'")

except Exception as e:
    print(f"Error during model training: {e}")
    print("Could not train model successfully.")


Building and training the model...
Model trained successfully!

Model accuracy: 0.74

Classification Report:
                      precision    recall  f1-score   support

             Anxiety       0.77      0.68      0.72       778
             Bipolar       0.81      0.64      0.72       575
          Depression       0.69      0.72      0.70      3081
              Normal       0.79      0.96      0.86      3270
Personality disorder       0.83      0.36      0.50       240
              Stress       0.71      0.37      0.48       534
            Suicidal       0.68      0.61      0.64      2131

            accuracy                           0.74     10609
           macro avg       0.75      0.62      0.66     10609
        weighted avg       0.73      0.74      0.73     10609

Model saved as 'mental_health_model.pkl'


In [ ]:
#Mental Health Resources Dictionary
# These resources will be provided based on the predicted mental health status

mental_health_resources = {
    'Normal': [
        "Maintaining mental wellness is important. Consider regular exercise, adequate sleep, and social connections.",
        "Resources: Mental Health America (www.mhanational.org) for wellness tips.",
        "Practice mindfulness and gratitude to maintain positive mental health."
    ],
    'Depression': [
        "It's okay to seek help for depression. You're not alone.",
        "Resources: National Institute of Mental Health (www.nimh.nih.gov), Crisis Text Line (Text HOME to 741741)",
        "Consider speaking with a mental health professional who can provide proper diagnosis and treatment.",
        "Small steps like regular physical activity and social connection can help alongside professional treatment."
    ],
    'Suicidal': [
        "If you're having thoughts of suicide, please reach out for immediate help.",
        "National Suicide Prevention Lifeline: 988 or 1-800-273-8255 (available 24/7)",
        "Crisis Text Line: Text HOME to 741741",
        "Please consider going to your nearest emergency room or calling emergency services.",
        "You matter and people care about you. Help is available."
    ],
    'Anxiety': [
        "Anxiety is common and treatable. Deep breathing and mindfulness can help in the moment.",
        "Resources: Anxiety and Depression Association of America (www.adaa.org)",
        "Consider speaking with a mental health professional about therapy options like CBT.",
        "The DARE approach (Defuse, Allow, Run toward, Engage) can be helpful for anxiety management."
    ],
    'Stress': [
        "Stress management techniques include exercise, meditation, and setting boundaries.",
        "Resources: American Psychological Association (www.apa.org/topics/stress)",
        "Consider time management strategies and prioritizing self-care.",
        "Progressive muscle relaxation and guided imagery can help reduce physical stress symptoms."
    ],
    'Bi-Polar': [
        "Bipolar disorder is treatable with proper medical care.",
        "Resources: Depression and Bipolar Support Alliance (www.dbsalliance.org)",
        "Please consult with a psychiatrist for proper diagnosis and treatment options.",
        "Maintaining a regular sleep schedule and mood tracking can help alongside professional treatment."
    ],
    'Personality Disorder': [
        "Personality disorders are complex but treatable with professional help.",
        "Resources: National Education Alliance for Borderline Personality Disorder (www.borderlinepersonalitydisorder.org)",
        "Specialized therapies like DBT can be effective for many personality disorders.",
        "Building a strong therapeutic relationship with a mental health professional is key to treatment."
    ]
}

In [ ]:
#Chatbot Interface

def get_resources(prediction):
    """Return appropriate resources based on prediction
    takes the prediction as input
    and returns the corresponding resources from the mental_health_resources dictionary.
    """
    return mental_health_resources.get(prediction, ["Please consult with a mental health professional for support."])

def safe_predict(text, model):
    """Make a prediction with error handling
    this function takes user input, preprocesses it, and
    uses the trained model to make a prediction.
    It also provides confidence scores for the predictions.
    If any error occurs, it prints the error message and returns
    "Unable to classify" with a confidence of 0.
    """
    try:
        # Preprocess the input text
        processed_text = preprocess_text(text)

        # Make prediction
        prediction = model.predict([processed_text])[0]
        confidence_scores = model.predict_proba([processed_text])[0]
        max_confidence = max(confidence_scores)

        return prediction, max_confidence
    except Exception as e:
        print(f"Error during prediction: {e}")
        return "Unable to classify", 0.0

def display_disclaimer():
    """Display important disclaimer about the chatbot"""
    print("\n" + "="*80)
    print("IMPORTANT DISCLAIMER:")
    print("This chatbot is NOT a substitute for professional mental health diagnosis or treatment.")
    print("It uses machine learning to analyze text patterns and provide general resources,")
    print("but it cannot provide medical advice or clinical diagnosis.")
    print("If you're experiencing a mental health crisis, please contact emergency services")
    print("or call the National Suicide Prevention Lifeline at 988 or 1-800-273-8255.")
    print("="*80 + "\n")

def mental_health_chatbot():
    """Main function for the mental health support chatbot"""

    display_disclaimer()

    print("Welcome to the Mental Health Support Chatbot!")
    print("You can share how you're feeling, and I'll try to provide helpful resources.")
    print("Type 'exit' at any time to end the conversation.\n")

    # Load the trained model
    try:
        model = joblib.load('mental_health_model.pkl')
        print("Model loaded successfully!\n")
    except:
        print("Error loading the model. Using the model trained in this session.\n")
        model = model_pipeline

    while True:
        user_input = input("How are you feeling today? (or type 'exit' to quit): ")

        if user_input.lower() == 'exit':
            print("\nThank you for using the Mental Health Support Chatbot.")
            print("Remember, it's always okay to seek professional help.")
            break

        if not user_input.strip():
            print("I notice you didn't type anything. Would you like to try again?")
            continue

        prediction, confidence = safe_predict(user_input, model)

        print(f"\nBased on your message, I'm detecting patterns that might be related to: {prediction}")

        if confidence < 0.5:
            print("(Note: This is a low-confidence assessment and may not be accurate)")

        print("\nHere are some resources that might be helpful:")
        for resource in get_resources(prediction):
            print(f"- {resource}")

        if prediction == 'Suicidal':
            print("\nIMPORTANT: If you're having thoughts of harming yourself,")
            print("please call the National Suicide Prevention Lifeline immediately:")
            print("988 or 1-800-273-8255 (available 24/7)")

        print("\nRemember: This chatbot can't provide medical advice or diagnosis.")
        print("It's always best to consult with a mental health professional.\n")

In [ ]:
#Run the chatbot
if __name__ == "__main__":
    mental_health_chatbot()


IMPORTANT DISCLAIMER:
This chatbot is NOT a substitute for professional mental health diagnosis or treatment.
It uses machine learning to analyze text patterns and provide general resources,
but it cannot provide medical advice or clinical diagnosis.
If you're experiencing a mental health crisis, please contact emergency services
or call the National Suicide Prevention Lifeline at 988 or 1-800-273-8255.

Welcome to the Mental Health Support Chatbot!
You can share how you're feeling, and I'll try to provide helpful resources.
Type 'exit' at any time to end the conversation.

Model loaded successfully!


Based on your message, I'm detecting patterns that might be related to: Normal

Here are some resources that might be helpful:
- Maintaining mental wellness is important. Consider regular exercise, adequate sleep, and social connections.
- Resources: Mental Health America (www.mhanational.org) for wellness tips.
- Practice mindfulness and gratitude to maintain positive mental health.

R